# Setup

(Mostly taken from TransformerLens main demo. No need to read)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")
%pip install kaleido

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-zvl15ix7
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-zvl15ix7
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
%%capture
!pip install kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [ ]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Nisch")

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

# from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from einops import rearrange

# from sklearn.decomposition import PCA
from scipy.special import softmax

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache, loading_from_pretrained
from transformer_lens.loading_from_pretrained import get_checkpoint_labels, get_official_model_name
torch.set_grad_enabled(False)

In [ ]:
from transformer_lens import evals
import matplotlib.pyplot as plt
import collections
import plotly.graph_objects as go

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# useful for sanity checks
model = HookedTransformer.from_pretrained("attn-only-2l", device=device)
# model3 = HookedTransformer.from_pretrained("attn-only-3l",device=device)


Loaded pretrained model attn-only-2l into HookedTransformer


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def plot_explained_variance(expl_variance_ratio):
    exp_var = np.cumsum(expl_variance_ratio) * 100

    plt.figure(figsize=(3, 3))
    plt.plot(range(1, len(exp_var)+1), exp_var)
    plt.ylim([0, 105])
    plt.xlabel("Eigen-vectors")
    plt.ylabel("Explained variance")
    plt.show()

In [ ]:
def bhattacharyya_distance_matrix(mat):
    """
    Takes a collection of models and computes the pairwise
    Bhattacharyya distance between all pairs of models.
    """
    mat = np.sqrt(mat) + 1e-9
    mat1 = np.transpose(mat, axes=[1, 0, 2])
    mat2 = np.transpose(mat, axes=[1, 2, 0])

    Dmat = 0.0
    dim = len(mat1)
    batch = 500

    # Compute distance in batches to avoid OOM
    for i in range(0, dim, batch):
        Dmat += (np.log(mat1[i:i+batch] @ mat2[i:i+batch])).sum(0)
    Dmat = Dmat / dim
    return Dmat

def compute_inpca(Dmat):
    """
    Compute the InPCA embedding from a pairwise distance matrix
    """
    # Double center matrix
    ldim = Dmat.shape[0]
    Pmat = np.eye(ldim) - 1.0/ ldim
    Wmat = (Pmat @ Dmat @ Pmat) / 2

    eigenval, eigenvec = np.linalg.eigh(Wmat)

    #Sort eigen-values by magnitude
    sort_ind = np.argsort(-np.abs(eigenval))
    eigenval = eigenval[sort_ind]
    eigenvec = eigenvec[:, sort_ind]
    sqrt_eigenval = np.sqrt(np.abs(eigenval))

    # Find projections
    projection = eigenvec * sqrt_eigenval.reshape(1, -1)

    return eigenval, projection



In [ ]:
model2 = HookedTransformer.from_pretrained("stanford-gpt2-small-a", device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model stanford-gpt2-small-a into HookedTransformer


# Weight and Activity dimensionlity analysis

## PCA

This is an approach to visualize the trajectory of the network during training in the functional space. The idea was first dervied and discussed here: https://www.pnas.org/doi/10.1073/pnas.1817218116
Curcially it involves calculating a negative curvature distance (bhattacharya) and using that instead of the euclidean matriz for PCA. It provides pretty cool visualizations of trajectory differences.

In [ ]:
# Small batch size to avoid cuda memory issues on colab
pile_batch_size = 1
pile_dataloader = evals.make_pile_data_loader(tokenizer=model.tokenizer, batch_size=pile_batch_size)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

10000


Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
token = next(iter(pile_dataloader))['tokens'].to(device)
loss_vec, cache1 = model.run_with_cache(token, return_type='loss', loss_per_token=True, remove_batch_dim = True)
loss_vec, cache2 = model2.run_with_cache(token, return_type='loss', loss_per_token=True, remove_batch_dim = True)

In [ ]:
dir(model)
print(model.W_U.shape, model2.W_U.shape)
print(cache1['ln_final.hook_normalized'].shape)
print(cache2['ln_final.hook_normalized'].shape)

NameError: ignored

In [ ]:
def inpca(model_predictions):
  predictions = model_predictions.to('cpu').detach().numpy()
  probabilities = softmax(predictions, axis=2)
  distance_matrix = bhattacharyya_distance_matrix(probabilities)
  eigenval, embed = compute_inpca(distance_matrix)
  embed = embed.reshape([4, len(checkpoint_indices), -1])
  # embed = embed.reshape([3, time_length, -1])
  return embed

def in_context_learning_score(model, tokens):
    loss_vec = model(tokens, return_type='loss', loss_per_token=True)
    return (loss_vec[..., 500] - loss_vec[..., 50]).mean()

In [ ]:
#Step 1: get model internal activities, mixed models
with torch.no_grad():
    torch.cuda.empty_cache()
checkpoint_indices = [10, 25, 35, 60, -1] # for quick-check
# checkpoint_indices = [0, 5, 10, 20, 25, 35, 40, 60, 100, -1]
time_length = 300
token = next(iter(pile_dataloader))['tokens'][:,:time_length].to(device)
model_predictions = []
model_to_tokens_trained_on = []
proj_dim = 264

for model_name in ["attn-only-1l", "attn-only-2l", "stanford-gpt2-small-a"]:
    print(model_name)
    tokens_trained_on = []
    model_porjection = []
    for index in checkpoint_indices:
      model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

      tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
      tokens_trained_on.append(tokens_seen_for_this_checkpoint)

      loss_vec, cache = model.run_with_cache(token, return_type='loss', loss_per_token=True)

      model_logits.append(logits_for_checkpoint)
    model_predictions.append(torch.stack(model_logits, dim=0).squeeze())
    model_to_tokens_trained_on.append(tokens_trained_on)

with torch.no_grad():
    torch.cuda.empty_cache()


In [ ]:
#Step 1: get model internal activities
with torch.no_grad():
    torch.cuda.empty_cache()
# checkpoint_indices = [10, 25, 35, 60, -1] # for quick-check
checkpoint_indices = [0, 5, 10, 20, 25, 35, 40, 60, 100, -1]
time_length = 300
token = next(iter(pile_dataloader))['tokens'][:,:time_length].to(device)
model_predictions = []
model_to_tokens_trained_on = []

for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l", "attn-only-4l"]:
    print(model_name)
    tokens_trained_on = []
    model_logits = []
    for index in checkpoint_indices:
      model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

      tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
      tokens_trained_on.append(tokens_seen_for_this_checkpoint)

      logits_for_checkpoint = model_for_this_checkpoint(token, return_type="logits")
      model_logits.append(logits_for_checkpoint)
    model_predictions.append(torch.stack(model_logits, dim=0).squeeze())
    model_to_tokens_trained_on.append(tokens_trained_on)

with torch.no_grad():
    torch.cuda.empty_cache()


In [ ]:
#Step 2: Reshape it
model_predictions = rearrange(model_predictions, 'd0 d1 d2 d3 -> (d0 d1) d2 d3')
print(model_predictions.shape) # torch.Size([3Xcheckpoints, time_length, 48262])

In [ ]:
#Step 3
with torch.no_grad():
    torch.cuda.empty_cache()
pca = inpca(model_predictions)
print(pca.shape) # (3XcheckpointsXdims)

In [ ]:
for j in range(5):
  fig = go.Figure(layout={'title': f'PC {j+1}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
  model_names = ["attn-only-1l", "attn-only-2l", "attn-only-3l", "attn-only-4l"]
  for i, data in enumerate(pca):
    tokens_trained_on = model_to_tokens_trained_on[i]
    fig.add_trace(go.Scatter(x=tokens_trained_on, y=data[:,j], name=model_names[i]))
  fig.show()
  fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/ipca_{j+1}_probabilities.png", scale=5)


## Weights Analysis

The idea here is to look at the evolution of the spectral entropy over training time. It can be thought of as tracking the feature learning property of the weight matrices, and it aligns phenomenologically with appearance of induction heads. The idea is based on t the work: https://arxiv.org/pdf/1312.6120.pdf

In [ ]:
def weight_dimensions(model):
    q_ov = model.OV.svd()[1].flatten()
    q_qk = model.QK.svd()[1].flatten()
    eff_dim_ov = (q_ov).sum()**2 / (q_ov**2).sum()
    eff_dim_qk = (q_qk).sum()**2 / (q_qk**2).sum()
    dims = [eff_dim_ov, eff_dim_qk]
    return dims

In [ ]:
def weight_dimensions_unroll(model):
    q_ov = model.OV.svd()[1]
    q_qk = model.QK.svd()[1]
    eff_dim_ov = (q_ov).sum(axis=-1)**2 / (q_ov**2).sum(axis=-1)
    eff_dim_qk = (q_qk).sum(axis=-1)**2 / (q_qk**2).sum(axis=-1)
    dims = [eff_dim_ov.mean(axis=-1), eff_dim_qk.mean(axis=-1)]
    return dims

In [ ]:
def effective_rank(model):
  q_ov = model.OV.svd()[1]
  q_qk = model.QK.svd()[1]
  q_ov = q_ov/q_ov.sum(dim=-1, keepdim=True)
  q_qk = q_qk/q_qk.sum(dim=-1, keepdim=True)
  q_ov = torch.sum(q_ov * torch.log(q_ov), dim=-1)
  q_qk = torch.sum(q_qk * torch.log(q_qk), dim=-1)
  dims = [torch.exp(-q_ov).mean(axis=-1).cpu().detach().numpy(), torch.exp(-q_qk).mean(axis=-1).cpu().detach().numpy()]
  del q_ov, q_qk
  torch.cuda.empty_cache()
  return dims

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()
# checkpoint_indices = [10, 25, 35, 60, -1]
checkpoint_indices = [0, 5, 10, 20, 25, 35, 40, 50, 60, 80, 100, 120, 140, -1] #pythia
# checkpoint_indices = [0, 5, 10, 20, 30, 40, 80, 100, 200, 300, -200, -100, -1]
model_to_effective_dimension_ov = {}
model_to_effective_dimension_qk = {}
model_to_tokens_trained_on = {}

# for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l", "attn-only-4l"]:
for model_name in ["pythia-160m"]: #, "stanford-gpt2-medium-a"]:
    print(model_name)
    tokens_trained_on = []
    effective_dimension_ov =[]
    effective_dimension_qk =[]
    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        # temp_ov, temp_qk = weight_dimensions(model_for_this_checkpoint)
        # temp_ov, temp_qk = weight_dimensions_unroll(model_for_this_checkpoint)
        temp_ov, temp_qk = effective_rank(model_for_this_checkpoint)

        # effective_dimension_ov.append(temp_ov.item())
        # effective_dimension_qk.append(temp_qk.item())
        effective_dimension_ov.append(temp_ov)
        effective_dimension_qk.append(temp_qk)

        del temp_ov, temp_qk, model_for_this_checkpoint
        torch.cuda.empty_cache()

    model_to_tokens_trained_on[model_name] = tokens_trained_on
    model_to_effective_dimension_ov[model_name] = effective_dimension_ov
    model_to_effective_dimension_qk[model_name] = effective_dimension_qk

In [ ]:
interp = {0:"encoding", 1:"decoding"}
for j in [0, 1]:
  fig = go.Figure(layout={f'title': f'Weights QK - {interp[j]}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  # fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
  for i, model_name in enumerate(model_to_effective_dimension_qk):
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    effective_dimension = np.asarray(model_to_effective_dimension_qk[model_name])
    if j == 0:
      subarray = effective_dimension[:,:-1].T if len(effective_dimension) > 1 else effective_dimension.T
      for c, y_val in enumerate(subarray):
        fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
    if j == 1 and i>0:
      fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,-1], name=f'{model_name} - last'))
    if i==0: fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,0], name=f'{model_name}'))
  fig.show()
  # fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/{interp[j]}_weights_qk_dim.png", scale=4)

interp = {0:"encoding", 1:"decoding"}
for j in [0, 1]:
  fig = go.Figure(layout={f'title': f'Weights OV - {interp[j]}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  # fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
  for i, model_name in enumerate(model_to_effective_dimension_ov):
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    effective_dimension = np.asarray(model_to_effective_dimension_ov[model_name])
    if j == 0:
      subarray = effective_dimension[:,:-1].T if len(effective_dimension) > 1 else effective_dimension.T
      for c, y_val in enumerate(subarray):
        fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
    if j == 1 and i>0:
      fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,-1], name=f'{model_name} - last'))
    if i==0: fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,0], name=f'{model_name}'))
  fig.show()
  # fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/{interp[j]}_weights_ov_dim.png", scale=4)



In [ ]:
fig = go.Figure(layout={f'title': f'Weights QK'})
fig.update_xaxes(title="Elapsed Training Tokens")
# fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
for i, model_name in enumerate(model_to_effective_dimension_qk):
  tokens_trained_on = model_to_tokens_trained_on[model_name]
  effective_dimension = np.asarray(model_to_effective_dimension_qk[model_name])
  for c, y_val in enumerate(effective_dimension.T):
    fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
  fig.show()
  fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/{model_name}_weights_qk_dim.png", scale=4)

fig = go.Figure(layout={f'title': f'Weights OV'})
fig.update_xaxes(title="Elapsed Training Tokens")
# fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
for i, model_name in enumerate(model_to_effective_dimension_ov):
  tokens_trained_on = model_to_tokens_trained_on[model_name]
  effective_dimension = np.asarray(model_to_effective_dimension_ov[model_name])
  for c, y_val in enumerate(effective_dimension.T):
    fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
  fig.show()
  fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/{model_name}_weights_ov_dim.png", scale=4)



## Activity Analysis

This part of analysing the activation vector is a more traditional comp neuro aproach. We basically look at the participation ration (effective dimensionality) of the activation vector, the idea being that learning translates to some effective lower dimensional structure being learned. This is part with most issue analysis wise since we want to keep huge amount of internal activation and analyse their internal correlations.


In [ ]:
# lens = ['attn.hook_q', 'attn.hook_k', 'attn.hook_v', 'attn.hook_attn_scores', 'attn.hook_pattern', 'attn.hook_z', 'hook_attn_out', 'hook_resid_post' ]
# lens = ['hook_attn_out', 'hook_resid_post' ]
def activity_eff_dim(model, tokens):
    loss_vec, cache = model.run_with_cache(tokens, return_type='loss', loss_per_token=True, remove_batch_dim = True)
    eff_dim = []
    temp_neuron =[]
    # for i in range(model.cfg.n_layers):
    #     temp_neuron.append(cache[f"blocks.{i}.hook_attn_out"].T)
    # neuron = torch.concatenate(temp_neuron, axis=0)
    for i in range(model.cfg.n_layers):
      neuron = cache[f"blocks.{i}.hook_attn_out"].T
      q = torch.linalg.svdvals(neuron @ neuron.T)
      temp_eff_dim = (q).sum()**2 / (q**2).sum()
      eff_dim.append(temp_eff_dim.item())
    return np.asarray(eff_dim)



In [ ]:
pile_batch_size = 1
pile_dataloader = evals.make_pile_data_loader(tokenizer=model.tokenizer, batch_size=pile_batch_size)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

10000


Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

# checkpoint_indices = [10, 25, 35, 60, -1]
# checkpoint_indices = [0, 5, 10, 20, 25, 35, 40, 50, 60, 80, 100, 120, 140, -1]
checkpoint_indices = [0, 5, 10, 20, 30, 40, 80, 100, 200, 300, -200, -100, -1] #stanford

model_to_in_context_learning_scores = {}
model_to_effective_dimension = {}
model_to_tokens_trained_on = {}

# for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l", "attn-only-4l"]:
# for model_name in ["pythia-70m", "pythia-160m"]:
for model_name in ["stanford-gpt2-medium-a"]:
    print(model_name)
    tokens_trained_on = []
    effective_dimension =[]

    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        effective_dimension_for_this_checkpoint = np.zeros(model_for_this_checkpoint.cfg.n_layers)
        num_batches = 100 // pile_batch_size   # Use subset of dataset for the sake of time
        for i, x in enumerate(pile_dataloader):
            tokens = x['tokens'].to(device)
            effective_dimension_for_this_checkpoint += activity_eff_dim(model_for_this_checkpoint, tokens)
            if i == num_batches:
                break
        effective_dimension.append(effective_dimension_for_this_checkpoint/num_batches)

    model_to_tokens_trained_on[model_name] = tokens_trained_on
    model_to_effective_dimension[model_name] = effective_dimension
    with torch.no_grad():
      torch.cuda.empty_cache()


stanford-gpt2-medium-a


Using pad_token, but it is not set yet.


Loaded pretrained model stanford-gpt2-medium-a into HookedTransformer


In [ ]:
# for i, model_name in enumerate(model_to_effective_dimension):
#   tokens_trained_on = model_to_tokens_trained_on[model_name]
#   effective_dimension = np.asarray(model_to_effective_dimension[model_name])

#   fig = go.Figure(layout={f'title': f'{model_name} Laywerwise - transfromer output dimension'})
#   fig.update_xaxes(title="Elapsed Training Tokens", type='log')
#   fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
#   for j in range(i+1):
#     print(effective_dimension.shape)
#     fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,j], name=f'Layer {j+1}'))
#   fig.show()
# fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/residual_activity_dim.png", scale=4)
interp = {0:"encoding", 1:"decoding"}
for j in [0, 1]:
  fig = go.Figure(layout={f'title': f'Laywerwise - transfromer {interp[j]}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
  for i, model_name in enumerate(model_to_effective_dimension):
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    effective_dimension = np.asarray(model_to_effective_dimension[model_name])
    if j == 0:
      subarray = effective_dimension[:,:-1].T if len(effective_dimension) > 1 else effective_dimension.T
      for c, y_val in enumerate(subarray):
        fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
    if j == 1 and i>0:
      fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,-1], name=f'{model_name} - last'))
    if i==0: fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,0], name=f'{model_name}'))
  fig.show()
  # fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/trasnsformer_{interp[j]}_activity_dim.png", scale=4)


# fig = go.Figure(layout={'title': 'Activity dimension - Residual Changes'})
# fig.update_xaxes(title="Elapsed Training Tokens", type='log')
# fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
# for model_name in model_to_in_context_learning_scores:
#   in_context_learning_scores = model_to_in_context_learning_scores[model_name]
#   effective_dimension = np.asarray(model_to_effective_dimension[model_name])
#   tokens_trained_on = model_to_tokens_trained_on[model_name]
#   fig.add_trace(go.Scatter(x=tokens_trained_on, y=effective_dimension[:,1], name=model_name))
# fig.show()
# # fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/residual_activity_dim.png", scale=4)



In [ ]:
# fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
for i, model_name in enumerate(model_to_effective_dimension):
  fig = go.Figure(layout={f'title': f'Laywerwise - transfromer {model_name}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  tokens_trained_on = model_to_tokens_trained_on[model_name]
  effective_dimension = np.asarray(model_to_effective_dimension[model_name])
  for c, y_val in enumerate(effective_dimension.T):
    fig.add_trace(go.Scatter(x=tokens_trained_on, y=y_val, name=f'{model_name} - {c+1}'))
  fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/{model_name}_transformer__dim.png", scale=4)
  fig.show()

## Token-time Dimension

Incomplete work: tentative idea to analyse the structure through the context window.

In [ ]:
temp_model1 = HookedTransformer.from_pretrained("attn-only-1l", device=device)
temp_model2 = HookedTransformer.from_pretrained("attn-only-2l", device=device)
temp_model3 = HookedTransformer.from_pretrained("attn-only-3l", device=device)

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer


In [ ]:
# Small batch size to avoid cuda memory issues on colab
# lens = ['hook_attn_out', 'hook_resid_post' ]
pile_batch_size = 5
pile_dataloader = evals.make_pile_data_loader(tokenizer=model.tokenizer, batch_size=pile_batch_size)
def context_dim(model, j):
  x = []
  for i in range(50): # make it 50
    token = next(iter(pile_dataloader))['tokens'].to(device)
    loss_vec, cache = model.run_with_cache(token, return_type='loss', loss_per_token=True)
    x.append(cache[f"blocks.{j}.hook_resid_post"]) #hook_attn_out
  x = torch.cat(x, axis=0)
  # Reshaping to [TxNxS]
  x = x.permute(1, 2, 0)
  # Calculating covariance for each T, resulting in [TxNxN] array
  # covariance_matrices = torch.matmul(x, x.transpose(2, 1))
  covariance_matrices = torch.matmul(x.transpose(2, 1), x)

  with torch.no_grad():
    torch.cuda.empty_cache()
  singular_values = torch.svd(covariance_matrices).S
  participation_ratio = (singular_values.sum(dim=-1) ** 2) / (singular_values ** 2).sum(dim=-1)

  return participation_ratio

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

10000


Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# pile_dataloader = evals.make_pile_data_loader(tokenizer=model.tokenizer, batch_size=pile_batch_size)
p_r = context_dim(temp_model3, 0)
print(p_r.shape)
with torch.no_grad():
    torch.cuda.empty_cache()

torch.Size([1024])


In [ ]:
fig = go.Figure()
fig = go.Figure(layout={'title': 'Transfromer output'})
fig.update_xaxes(title="Input Tokens")
fig.add_trace(go.Scatter(x=np.arange(1024), y=p_r.cpu()))
fig.show()
# fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/1L_tr0_token_time_dim.png", scale=4)

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

checkpoint_indices = [10, 35, 120, -1]
model_to_transformer_dimension = {}
model_to_tokens_trained_on = {}

for i, model_name in enumerate(["attn-only-1l", "attn-only-2l", "attn-only-3l"]):
    print(model_name)
    tokens_trained_on = []
    transformer_dimension=[]

    for index in checkpoint_indices:
        transformer_dimension_per_layer =[]
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        for j in range(i+1):
          t_dim = context_dim(model_for_this_checkpoint, j)
          transformer_dimension_per_layer.append(t_dim.cpu())
          with torch.no_grad():
            torch.cuda.empty_cache()
        transformer_dimension.append(transformer_dimension_per_layer)

    model_to_transformer_dimension[model_name] = transformer_dimension
    model_to_tokens_trained_on[model_name] = tokens_trained_on


attn-only-1l


Loaded pretrained model attn-only-1l into HookedTransformer


Loaded pretrained model attn-only-1l into HookedTransformer


Loaded pretrained model attn-only-1l into HookedTransformer


Loaded pretrained model attn-only-1l into HookedTransformer
attn-only-2l


...

Loaded pretrained model attn-only-2l into HookedTransformer
attn-only-3l


Loaded pretrained model attn-only-3l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer


In [ ]:
window_size = 10
kernel = np.ones(window_size) / window_size
fig = go.Figure()
fig.update_xaxes(title="Token Time")
for i, model_name in enumerate(model_to_transformer_dimension):
  transformer_dimension = model_to_transformer_dimension[model_name]
  tokens_trained_on = model_to_tokens_trained_on[model_name]
  for j in range(i+1):
    fig = go.Figure()
    fig = go.Figure(layout={'title': f'{model_name}: layer {j+1}'})
    fig.update_xaxes(title="Input Tokens")
    fig.update_yaxes(title="Dim")
    for tokens, transformer_dimension_context in zip(tokens_trained_on, transformer_dimension):
      y_val = transformer_dimension_context[j]
      y_hat = np.convolve(y_val, kernel, mode='same')
      fig.add_trace(go.Scatter(x=np.arange(1024), y=y_hat, name=f'{tokens:.1e}', line_shape='spline'))
    # fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/transformer_tokenwise_dim_{model_name}_layer{j}.png", scale=4)
    fig.show()


## Spectral learning

Incomplete: But there are some predictions on two regimens of learning, first learning the relevant directions (eigen vectors) and then learning to associate the right eigen values to them. We are interested in the second part, and want to track this evolution with prediction being time scale separation of the evolution (with larger modes evolving and saturating quicker)

In [ ]:
# x = []
# for i in range(100):
#   token = next(iter(pile_dataloader))['tokens'].to(device)
#   loss_vec, cache = temp_model3.run_with_cache(token, return_type='loss', loss_per_token=True)
#   x.append(cache[f"blocks.0.hook_attn_out"])

# print(len(x), x[0].shape)
# x = torch.cat(x, axis=0)
# print(x.shape)

# print(x.shape)
# # Reshaping to [TxNxS]
# x = x.permute(1, 2, 0)
# print(x.shape)
# # Calculating covariance for each T, resulting in [TxNxN] array
# covariance_matrices = torch.matmul(x, x.transpose(2, 1))
# with torch.no_grad():
#     torch.cuda.empty_cache()
# print(covariance_matrices.shape)
# singular_values = torch.svd(covariance_matrices).S
# print(singular_values.shape)
# participation_ratio = (singular_values.sum(dim=-1) ** 2) / (singular_values ** 2).sum(dim=-1)
# print(participation_ratio.shape)


In [ ]:
def weight_dimensions(model):
    q_ov = model.OV.svd()[1].flatten()
    q_qk = model.QK.svd()[1].flatten()
    eff_dim_ov = (q_ov).sum()**2 / (q_ov**2).sum()
    eff_dim_qk = (q_qk).sum()**2 / (q_qk**2).sum()
    dims = [eff_dim_ov, eff_dim_qk]
    return dims

In [ ]:
final_model = HookedTransformer.from_pretrained("attn-only-3l", checkpoint_index=-1, device=device)

Loaded pretrained model attn-only-3l into HookedTransformer


In [ ]:
svd = final_model.OV.svd()
U = svd[0]
s= svd[1]
V = svd[2]
V_inv = torch.linalg.pinv(V)
U_inv = torch.linalg.pinv(U)
print(U.shape, s.shape, V.shape)

torch.Size([3, 8, 512, 64]) torch.Size([3, 8, 64]) torch.Size([3, 8, 512, 64])


In [ ]:
result_einsum = torch.einsum('ijkl,ijlm,ijmn->ijkn', V.transpose(-2, -1), M.AB, U)

In [ ]:
temp_model = HookedTransformer.from_pretrained("attn-only-3l", checkpoint_index=-1, device=device)
M = temp_model.OV
result_einsum = torch.einsum('ijkl,ijlm,ijmn->ijkn', U_inv, M.AB, V_inv.transpose(-2, -1))
result_diagonals = torch.einsum('ijkl->ijk', result_einsum)

Loaded pretrained model attn-only-3l into HookedTransformer


In [ ]:
(result_diagonals.cpu().numpy()).shape

(3, 8, 64)

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

checkpoint_indices = [0, 5, 10, 20, 25, 35, 40, 50, 60, 80, 100, 120, 140, -1]
# mode_spectrum_ov = {}
model_spectrum_qk = {}
model_to_tokens_trained_on = {}
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    print(model_name)
    tokens_trained_on = []
    # spectrum_ov =[]
    spectrum_qk =[]

    final_model = HookedTransformer.from_pretrained(model_name, checkpoint_index=-1, device=device)
    svd = final_model.QK.svd()
    U = svd[0]
    s= svd[1]
    V = svd[2]
    V_inv = torch.linalg.pinv(V)
    U_inv = torch.linalg.pinv(U)

    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        M = model_for_this_checkpoint.OV
        result_einsum = torch.einsum('ijkl,ijlm,ijmn->ijkn', U_inv, M.AB, V_inv.transpose(-2, -1))
        result_diagonals = torch.einsum('ijkl->ijk', result_einsum)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        spectrum_qk.append(result_diagonals.cpu().numpy())

    model_to_tokens_trained_on[model_name] = tokens_trained_on
    model_spectrum_qk[model_name] = spectrum_qk


attn-only-1l
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
attn-only-2l
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrai

In [ ]:
spectrum = model_spectrum_qk["attn-only-2l"]
spectrum = np.asarray(spectrum)
# spectrum = spectrum[:,2,6,:10].T # timeXlayersXheadXdim
print(spectrum.shape)
head_mean_spectrum = np.mean(spectrum, axis =2)
print(head_mean_spectrum.shape)
spectrum = head_mean_spectrum[:,1,:10].T # timeXlayersXdim
print(spectrum.shape)

(14, 2, 8, 64)
(14, 2, 64)
(10, 14)


In [ ]:
model_names = ["attn-only-1l", "attn-only-2l", "attn-only-3l"]
model_name = "attn-only-1l"
# for model_name in model_names:
for j in range(1):
  spectrum = model_spectrum_qk[model_name]
  spectrum = np.asarray(spectrum)
  head_mean_spectrum = np.mean(spectrum, axis =2)
  spectrum = head_mean_spectrum[:,j,:].T
  fig = go.Figure(layout={'title': f'Spectral dynamics: {model_name} - {j}'})
  fig.update_xaxes(title="Elapsed Training Tokens", type='log')
  fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
  for i, sp in enumerate(spectrum):
    tokens_trained_on = model_to_tokens_trained_on["attn-only-3l"]
    fig.add_trace(go.Scatter(x=tokens_trained_on, y=sp, name=i, opacity = 0.8))
  fig.show()
  fig.write_image(f"/content/gdrive/MyDrive/Colab Notebooks/qk_spectral_dynamics_{model_name}_layer_{j}.png", scale=5)


In [ ]:
# import plotly.subplots as sp
# import plotly.graph_objects as go

# fig = sp.make_subplots(rows=3, cols=8)

# for i in range(3):
#     for j in range(8):
#         heatmap = go.Heatmap(z=result_einsum[i, j].cpu().numpy(), showscale=False)
#         heatmap = go.Heatmap(z=result_einsum[i, j].cpu().numpy(), showscale=False)
#         fig.add_trace(heatmap, row=i+1, col=j+1)
#     break

# fig.update_layout(height=900, width=1600, title_text="Heatmaps of 3x8 matrices", showlegend=False)
# fig.show()


# Induction Heads form in phase change (Prefix Matching Score)
-- Not mine, copied from somehwere in github to just play with.. Everything here on and below is just replication of the anthropic work


In [ ]:
batch_size = 10
seq_len = 50
random_tokens = torch.randint(1000, 10000, (batch_size, seq_len)).to(model.cfg.device)
repeated_tokens = einops.repeat(random_tokens, "batch seq_len -> batch (2 seq_len)")
repeated_tokens[:, 0] = model.tokenizer.bos_token_id

In [ ]:
# hook copied from transformer lens main demo
def induction_score_hook(
    pattern: TT["batch", "head_index", "dest_pos", "source_pos"],
    hook: HookPoint,
):
    # We take the diagonal of attention paid from each destination position to source positions seq_len-1 tokens back
    # (This only has entries for tokens with index>=seq_len)
    induction_stripe = pattern.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
    # Get an average score per head
    induction_score = einops.reduce(induction_stripe, "batch head_index position -> head_index", "mean")
    # Store the result.
    induction_score_store[hook.layer(), :] = induction_score

# We make a boolean filter on activation names, that's true only on attention pattern names.
pattern_hook_names_filter = lambda name: name.endswith("pattern")

In [ ]:
checkpoint_indices = [10, 15, 20, 25, 30, 35, 40, 45, 50, 60, -1]
model_to_scores_per_layer_head = {}
model_to_tokens_trained_on = {}
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    tokens_trained_on = []
    induction_scores_per_layer_head = collections.defaultdict(list)
    for index in checkpoint_indices:
        # Load the model from the relevant checkpoint by index
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        # induction_score_hook will store results here
        induction_score_store = torch.zeros((model_for_this_checkpoint.cfg.n_layers, model_for_this_checkpoint.cfg.n_heads), device=model_for_this_checkpoint.cfg.device)

        model_for_this_checkpoint.run_with_hooks(
            repeated_tokens,
            return_type=None, # For efficiency, we don't need to calculate the logits
            fwd_hooks=[(
                pattern_hook_names_filter,
                induction_score_hook
            )]
        )

        for layer in range(model_for_this_checkpoint.cfg.n_layers):
            for head in range(model_for_this_checkpoint.cfg.n_heads):
                induction_scores_per_layer_head[str(layer) + ',' + str(head)].append(induction_score_store[layer][head].item())
    model_to_scores_per_layer_head[model_name] = induction_scores_per_layer_head
    model_to_tokens_trained_on[model_name] = tokens_trained_on

In [ ]:
for model_name in model_to_scores_per_layer_head:
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    scores_per_layer_head = model_to_scores_per_layer_head[model_name]
    fig = go.Figure(layout={'title': model_name})
    fig.update_xaxes(title="Elapsed Training Tokens", type='log')
    fig.update_yaxes(title="Prefix Matching Score")
    fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
    for layer_head, scores in scores_per_layer_head.items():
        fig.add_trace(go.Scatter(x=tokens_trained_on, y=scores, name=layer_head))
    fig.update_layout(yaxis_range=[0.0,1.0])
    fig.show()

# Loss Curves Diverge during Phase Change

In [ ]:
checkpoint_indices = [10, 15, 20, 25, 30, 35, 40, 45, 50, 60, -1]
model_to_loss_curve = {}
model_to_tokens_trained_on = {}
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    tokens_trained_on = []
    losses = []
    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        loss_for_this_checkpoint = 0
        num_batches = 40
        for i, x in enumerate(pile_dataloader):
            tokens = x['tokens'].to(device)
            loss_for_this_checkpoint += model_for_this_checkpoint(tokens, return_type='loss').item()
            if i == num_batches:
                break
        loss_for_this_checkpoint /= num_batches
        losses.append(loss_for_this_checkpoint)
    model_to_loss_curve[model_name] = losses
    model_to_tokens_trained_on[model_name] = tokens_trained_on

In [ ]:
for model_name in model_to_loss_curve:
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    losses = model_to_loss_curve[model_name]
    fig = px.line(x=tokens_trained_on, y=losses, title=model_name, labels={"x":"Elapsed Training Tokens", "y":"Loss (nats / token)"})
    fig.update_layout(yaxis_range=[2.0,8.0])
    fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
    fig.show()

Log x axis to see the phase change more clearly:

In [ ]:
for model_name in model_to_loss_curve:
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    losses = model_to_loss_curve[model_name]
    fig = px.line(x=tokens_trained_on, y=losses, title=model_name, labels={"x":"Elapsed Training Tokens", "y":"Loss (nats / token)"}, log_x=True)
    fig.update_layout(yaxis_range=[2.0,8.0])
    fig.add_vrect(x0=3e8, x1=1.5e9, line_width=1, fillcolor="gold", opacity=0.2)
    fig.show()

# Per Token Loss Principal component Analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [ ]:
# collect some examples
examples = []
num_examples = 200 // pile_batch_size
for i, x in enumerate(pile_dataloader):
    tokens = x['tokens'].to(device)
    examples.append(tokens)
    if i == num_examples - 1:
        break
examples[0].shape

torch.Size([4, 1024])

In [ ]:
indices = torch.randint(0, examples[0].shape[-1]-1, (len(examples) * pile_batch_size,))
indices.shape

torch.Size([200])

In [ ]:
checkpoint_indices = [10, 15, 20, 25, 30, 35, 40, 45, 50, 60, -1]
model_to_pca_features = {}
model_to_tokens_trained_on = {}
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    loss_data_matrix = torch.zeros((len(checkpoint_indices), len(examples) * pile_batch_size))
    tokens_trained_on = []
    for pos, index in enumerate(checkpoint_indices):
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        loss_vec_store = torch.zeros((len(examples) * pile_batch_size, examples[0].shape[-1]-1))
        for i, ex in enumerate(examples):
            loss_vec = model_for_this_checkpoint(ex, return_type="loss", loss_per_token=True)
            loss_vec_store[i*pile_batch_size:i*pile_batch_size + pile_batch_size] = loss_vec.cpu()
        loss_sampled = loss_vec_store[torch.arange(loss_vec_store.shape[0]), indices]
        # I needed to put this on cpu to avoid cuda memory errors...
        loss_data_matrix[pos] = loss_sampled.cpu()
    loss_data_scaled = StandardScaler().fit_transform(loss_data_matrix)
    pca_features = pca.fit_transform(loss_data_scaled)
    model_to_pca_features[model_name] = pca_features
    model_to_tokens_trained_on[model_name] = tokens_trained_on

In [ ]:
for model_name in model_to_pca_features:
    pca_features = model_to_pca_features[model_name]
    tokens_trained_on = model_to_tokens_trained_on[model_name]
    fig1 = go.Figure()
    for i in range(1, len(pca_features)):
        # color phase change window red
        line_color = "red" if 3e8 <= tokens_trained_on[i] <= 1.5e9 else 'blue'
        fig1.add_trace(go.Scatter(x=pca_features[i-1: i+1, 0],
                                    y=pca_features[i-1: i+1, 1],
                                    line={"width": 1, "dash": "dash", "color": line_color}, showlegend=False))
    fig1.update(layout_showlegend=False)

    fig2 = px.scatter(x=pca_features[:, 0], y=pca_features[:, 1], color=list(map(str, tokens_trained_on)))
    fig3 = go.Figure(data=fig1.data + fig2.data)
    fig3.update_layout(legend_title="Elapsed Training Tokens", title=model_name)
    fig3.show()

# B - A per token losses on Harry Potter

In [ ]:
context = """Mr. and Mrs. Dursley, of number four, Privet Drive, were
proud to say that they were perfectly normal, thank
you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t
hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which
made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin
and blonde and had nearly twice the usual amount of neck, which
came in very useful as she spent so much of her time craning over
garden fences, spying on the neighbors. The Dursleys had a small
son called Dudley and in their opinion there was no finer boy
anywhere.
The Dursleys had everything they wanted, but they also had a
secret, and their greatest fear was that somebody would discover it.
They didn’t think they could bear it if anyone found out about the
Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met
for several years; in fact, Mrs. Dursley pretended she didn’t have a
sister, because her sister and her good-for-nothing husband were
as unDursleyish as it was possible to be. The Dursleys shuddered
to think what the neighbors would say if the Potters arrived in the
street. The Dursleys knew that the Potters had a small son, too, but
they had never even seen him. This boy was another good reason
for keeping the Potters away; they didn’t want Dudley mixing with
a child like that.
"""

In [ ]:
# take indices right before and after phase change window (based on pca plot above)
a_index = 25
b_index = 50

model_before_phase_change = HookedTransformer.from_pretrained('attn-only-2l', device=device, checkpoint_index=a_index)
model_after_phase_change = HookedTransformer.from_pretrained('attn-only-2l', device=device, checkpoint_index=b_index)

In [ ]:
loss_vec_before = model_before_phase_change(context, return_type='loss', loss_per_token=True)
loss_vec_after = model_after_phase_change(context, return_type='loss', loss_per_token=True)

loss_vec_difference = loss_vec_after - loss_vec_before

In [ ]:
str_tokens = model_before_phase_change.to_str_tokens(context)
z = utils.to_numpy(loss_vec_difference.reshape(20, -1))
z_text = np.array(str_tokens[1:]).reshape(z.shape)

fig = px.imshow(z, color_continuous_midpoint=0.0, color_continuous_scale="RdBu", aspect="auto")
fig.update_traces(text=z_text, texttemplate="%{text}")
fig.show()

# Per-Token losses over training

In [ ]:
leys_idx = max(idx for idx, token in enumerate(str_tokens) if token == 'leys')
useful_idx = min(idx for idx, token in enumerate(str_tokens) if token == ' useful')

In [ ]:
checkpoint_indices = [10, 25, 35, 60, -1]
tokens_trained_on = []
leys_losses = []
useful_losses = []
mean_losses = []
for index in checkpoint_indices:
    model_for_this_checkpoint = HookedTransformer.from_pretrained('attn-only-2l', device=device, checkpoint_index=index)

    tokens_trained_on_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
    tokens_trained_on.append(tokens_trained_on_for_this_checkpoint)

    loss_vec = model_for_this_checkpoint(context, return_type="loss", loss_per_token=True)
    loss_for_leys = loss_vec[:, leys_idx-1].item()
    leys_losses.append(loss_for_leys)

    loss_for_useful = loss_vec[:, useful_idx-1].item()
    useful_losses.append(loss_for_useful)

    mean_losses.append(loss_vec.mean().item())

In [ ]:
fig = go.Figure()
fig.update_layout(yaxis_range=[0.0, 14.0])
fig.add_vrect(x0=3e8, x1=1.5e9, fillcolor='gold', line_width=1, opacity=0.2)
fig.update_xaxes(title="Elapsed Training Tokens")
fig.update_yaxes(title="Loss (nats / token)")
fig.add_trace(go.Scatter(x=tokens_trained_on, y=useful_losses, name=" useful"))
fig.add_trace(go.Scatter(x=tokens_trained_on, y=leys_losses, name="leys"))
fig.add_trace(go.Scatter(x=tokens_trained_on, y=mean_losses, name="mean loss", line=dict(color='gray', dash='dash')))
fig.show()